In [ ]:
def bidSubmissionFunctionForSCAToStorePB(tx_receipt, nonce,pb,flag):
    bid_submission_sol = w3.eth.contract(abi=abiA,address=tx_receipt.contractAddress)                     
    #  whenever we want to uipdate something we trsact that value so gas is used for that purpose we agin perfomr the transaction
    if flag == 'aucVar':
        call_fun = bid_submission_sol.functions.bidderEncryptedBidSubmission(pb).build_transaction({"chainId": chain_id, "from": my_addressA, "nonce": nonce + 1})
    else:#flag ===bidVar
        call_fun = bid_submission_sol.functions.updateBidderVariable(pb).build_transaction({"chainId": chain_id, "from": my_addressA, "nonce": nonce + 1})
        
    sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_keyA)
    tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.rawTransaction)
    tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)
def bidSubmissionFunctionForSCAToRetrievePB(tx_receipt,flag):
    # calling functions in contract blockand address
    #  this line call the the contract by passing the abi and adress of the transactoin

    bid_submission_sol = w3.eth.contract(abi=abiA, address=tx_receipt.contractAddress)
    
    # just viewing something from your transaction doesnt cost anyting
    if flag == 'aucVar':
        return bid_submission_sol.functions.retrieveBidderEncryptedBidSubmission().call()
    else:
        return bid_submission_sol.functions.fetchVariable().call()
      

In [ ]:
# this block imports all necessary libraries and a necessary thing is for this code to work, run ganache 
# and replace the address and publickeys as are required.

import numpy as np
import random as rd
from random import choice
import hashlib 
from phe import paillier
import random as rd
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import base64

# libs for rsa and others

from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization,hashes
from cryptography.hazmat.primitives.asymmetric import utils
import import_ipynb

# This code imports the web3 code notebook
import web3File
from web3File import *

def submitsPB(targetID, rdbid, rdnum, PBvalue, encryptionType,flag):

 if encryptionType == 'RSA':
        # Generate the pb12 and send to smart contract
        PBij = encrypt(str((dictionaryOfPublicKeys[targetID].raw_encrypt(rdbid + rdnum))).encode('utf8'), RSApublicKeyAuc)
        submittingPaddedData = generateSubmittingFormatData(PBij)

        values = bidSubmissionPhaseFunctiontoInitializeSCA()

        bidSubmissionFunctionForSCAToStorePB(values[0],values[1],submittingPaddedData,flag)

        return values[0]
 else:
        values = bidSubmissionPhaseFunctiontoInitializeSCA()
        bidSubmissionFunctionForSCAToStorePB(values[0],values[1],str(PBvalue),flag)
        return values[0]


def retrievesPB(tx_reciept,encryptionType,flag):
    if encryptionType == 'RSA':
        value = bidSubmissionFunctionForSCAToRetrievePB(tx_reciept,flag)
        
        value = generateRetrievingFormatData(value)

        value = int(decrypt(value, RSAprivateKeyAuc).decode('utf8'))

        return value  
    else:

        value = bidSubmissionFunctionForSCAToRetrievePB(tx_reciept,flag)
        return int(value)
        

def bidSubmission(flag,bidderHighest):

    bidder1,bidder2,b1rdnum,b1rdbid,b2rdnum,b2rdbid = generateTwoBiddersSTPCData(flag,bidderHighest)
    #here bidder i and bidder j generate stpc pairs first value of pbij and submit to the auctioneer via transaction reciept
    
    #trnasaction recipet for bidder i
    tx_reciepti =  submitsPB(bidder2, b1rdbid, b1rdnum, 0,'RSA','aucVar')
    
    #transaction recipet for bidder j
    tx_recieptj =  submitsPB(bidder1, b2rdbid, b2rdnum,0,'RSA','aucVar')
    
    #auctioneer retrives submitted pb from here 

    pb12 = retrievesPB(tx_reciepti,'RSA','aucVar')
    pb21 = retrievesPB(tx_recieptj,'RSA','aucVar')


    #auctioneer adds its random number fraction here

    m = find_addends(20000)
    k = find_addends(20000)

    # AUCTIONEER SUBMITS THESE VALUES TO THE BIDDER AFTER ADDING ONE ADDEND

    pb12 = np.dot(pb12, dictionaryOfPublicKeys[bidder2].raw_encrypt(m[0][0]))
    pb21 = np.dot(pb21, dictionaryOfPublicKeys[bidder1].raw_encrypt(k[0][0]))

    #from here auctioneer submits the PB to target bidders after partial randmom number addtion

    tx_reciepti = submitsPB(pb12,'pailier', 'bidVar')
    tx_recieptj = submitsPB(pb21,'pailier', 'bidVar')

    #bidder fetch the values of that have been updated s their private variable

    pb12 = retrievesPB(tx_reciepti,'bidVar')
    pb21 = retrievesPB(tx_recieptj,'bidVar')
    
    
    #BIDDER NOW ADD THE RANDOM RANDOM NUMBER TO THE OTHER BIDDERS DATA

    pb21 = dictionaryOfPrivateKeys[bidder1].raw_decrypt(pb21.item())
    pb12 = dictionaryOfPrivateKeys[bidder2].raw_decrypt(pb12.item())

    # BIDDER 1 ADDS THE RANDOM NUMBER TO PB12

    pb21 = pb21 + b1rdnum 
    pb21 = encrypt(str(pb21).encode(),RSApublicKeyAuc)
    
    # BIDDER 1 ADDS THE RANDOM NUMBER TO PB12

    pb12 = pb12 + b2rdnum
    pb12 = encrypt(str(pb12).encode(),RSApublicKeyAuc)

    # bOTH BIDDERS SUBMITS THEIR FINAL PB VALUE TO AUCTIONEER
    tx_reciepti = submitsPB(0,0,0, pb12, 'RSA', 'aucVar')
    tx_recieptj = submitsPB(0,0,0, pb21, 'RSA', 'aucVar')

    # aUCTIONEER NOW FETCHES BOTH pb VALUE AND ADD REMAINING ADDEND
    # auctioneer now adds other part of random addend to the pb values 

    pb12 = retrievesPB(tx_reciepti,'aucVar')
    pb21 = retrievesPB(tx_recieptj,'aucVar')

    pb12 = pb12 + m[0][1]
    pb21 = pb21 + k[0][1]

    return [pb12,pb21,bidder1,bidder2]
